# Loading

In [188]:
from Data.data_dicts import character_dict, source_dict, random_state

model_name = 'microsoft/DialoGPT-small'
character = 'Sheldon' # 'Barney' | 'Sheldon' | 'Harry' | 'Fry' | 'Vader' | 'Joey' | 'Phoebe' | 'Bender' | Default'
character_2 = 'Barney'
override_predictions = False

In [189]:
# Mount google drive
import os
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive',force_remount=True)
    base_folder = '/content/drive/My Drive/unibo/NLP_project/BarneyBot'
    os.system("pip install datasets")
    os.system("pip install transformers")
    os.system("pip install rouge_score")
    os.system("pip install -U sentence-transformers")
else:
    base_folder = os.getcwd()
    
in_folder = os.path.join(base_folder, 'Data', 'Characters', character)
if not os.path.exists(in_folder):
    os.makedirs(in_folder)
out_folder = os.path.join(base_folder, 'Data', 'Characters', character)
if not os.path.exists(out_folder):
    os.makedirs(out_folder)
    
in_folder_2 = os.path.join(base_folder, 'Data', 'Characters', character_2)
if not os.path.exists(in_folder_2):
    os.makedirs(in_folder_2)
out_folder_2 = os.path.join(base_folder, 'Data', 'Characters', character_2)
if not os.path.exists(out_folder_2):
    os.makedirs(out_folder_2)
    
in_folder_def = os.path.join(base_folder, 'Data', 'Characters', 'Default')
if not os.path.exists(in_folder_def):
    os.makedirs(in_folder_def)
out_folder_def = os.path.join(base_folder, 'Data', 'Characters', 'Default')
if not os.path.exists(out_folder_def):
    os.makedirs(out_folder_def)
    
metrics_folder = os.path.join(base_folder, 'Metrics')
if not os.path.exists(metrics_folder):
    os.makedirs(metrics_folder)

In [190]:
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
import json
import numpy as np
import time
import scipy as sp

In [191]:
def save_as_json(filepath, filename, data):
    if not os.path.exists(filepath):
        os.makedirs(filepath, exist_ok=True)
    with open(os.path.join(filepath, filename + ".json"), 'w') as f:
        f.write(json.dumps(data, indent=4))

def load_from_json(filepath, filename):
    if not os.path.exists(os.path.join(filepath, filename + '.json')):
        return dict()
    with open(os.path.join(filepath, filename + '.json'), 'r') as f:
        return json.load(f)

In [192]:
from datasets import load_dataset, DatasetDict

def load_df(character):
    dataset_path = os.path.join(base_folder, "Data", "Characters", character, character+'.csv')
    
    character_hg = load_dataset('csv', 
                                data_files=dataset_path, 
                                cache_dir=os.path.join(base_folder, "cache"))
    
    # 85% train / 10% test / 5% validation
    train_test_hg = character_hg['train'].train_test_split(test_size=0.15, seed=random_state)
    test_val = train_test_hg['test'].train_test_split(test_size=0.33, seed=random_state)
    
    
    character_hg = DatasetDict({
        'train': train_test_hg['train'],
        'test': test_val['train'],
        'val': test_val['test']
    })
    
    return character_hg

In [193]:
def construct_conv(row, tokenizer):
    MAX_LENGTH = 512
    row = list(reversed(list(row.values())))
    model_inputs = tokenizer(row)
    tokenizer_pad_token_id = tokenizer.encode('#')[0]
    for i in range(len(model_inputs['input_ids'])):
        model_inputs['input_ids'][i].append(tokenizer.eos_token_id)
        model_inputs['attention_mask'][i].append(1)
    model_inputs['input_ids'] = [item for sublist in model_inputs['input_ids'] for item in sublist]
    model_inputs['attention_mask'] = [item for sublist in model_inputs['attention_mask'] for item in sublist]
    if MAX_LENGTH > len(model_inputs['input_ids']):
        model_inputs['input_ids'] += [tokenizer_pad_token_id] * (MAX_LENGTH - len(model_inputs['input_ids']))
        model_inputs['attention_mask'] += [0] * (MAX_LENGTH - len(model_inputs['attention_mask']))
    elif MAX_LENGTH < len(model_inputs['input_ids']):
        model_inputs['input_ids'] = model_inputs['input_ids'][:MAX_LENGTH-1]
        model_inputs['input_ids'][-1] = tokenizer.eos_token_id
        model_inputs['attention_mask'] = model_inputs['attention_mask'][:MAX_LENGTH-1]
        model_inputs['attention_mask'][-1] = 1
    model_inputs["labels"] = model_inputs["input_ids"]
    return model_inputs

def preprocess_function(examples):
    tokenizer.pad_token = '#'
    model_inputs = construct_conv(examples, tokenizer)
    return model_inputs

In [194]:
os.environ["HF_DATASETS_CACHE"] = os.path.join(base_folder, "cache")
character_hg = load_df(character)

Using custom data configuration default-fbb5da8195eeafdf
Reusing dataset csv (C:\Users\david\Documents\unibo\natural_language_processing\project\BarneyBot\cache\csv\default-fbb5da8195eeafdf\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached split indices for dataset at C:\Users\david\Documents\unibo\natural_language_processing\project\BarneyBot\cache\csv\default-fbb5da8195eeafdf\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519\cache-ed070a17f87583b0.arrow and C:\Users\david\Documents\unibo\natural_language_processing\project\BarneyBot\cache\csv\default-fbb5da8195eeafdf\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519\cache-8b13a1828c5f655c.arrow
Loading cached split indices for dataset at C:\Users\david\Documents\unibo\natural_language_processing\project\BarneyBot\cache\csv\default-fbb5da8195eeafdf\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519\cache-2b4e70bae7b548bf.arrow and C:\Users\david\Documents\unibo\natural_language_processing\project\BarneyBot\cache\csv\default-fbb5da8195eeafdf\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519\cache-9730e803e8838750.arrow


In [195]:
checkpoint_folder = os.path.join(out_folder, character_dict[character]['checkpoint_folder'])
checkpoint_folder_2 = os.path.join(out_folder_2, character_dict[character_2]['checkpoint_folder'])

In [196]:
from transformers import TFAutoModelForCausalLM, AutoTokenizer, AdamWeightDecay

tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=os.path.join(base_folder, "cache"))
tokenizer.pad_token = '#'

model = TFAutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=checkpoint_folder)
model.compile(optimizer=AdamWeightDecay(learning_rate=2e-5))

model_2 = TFAutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=checkpoint_folder_2)
model_2.compile(optimizer=AdamWeightDecay(learning_rate=2e-5))

model_def = TFAutoModelForCausalLM.from_pretrained(model_name, cache_dir=os.path.join(base_folder, "cache"))
model_def.compile(optimizer=AdamWeightDecay(learning_rate=2e-5))

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at C:\Users\david\Documents\unibo\natural_language_processing\project\BarneyBot\Data\Characters\Sheldon\sheldon_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! Please ensure your labels are passed as keys in the input dict so that they are accessible to the model during the forward pass. To disable this behaviour, please pass a loss argument, or explicitly pass loss=None if you do not want your model to compute a loss.
All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialize

In [197]:
from transformers import DataCollatorForLanguageModeling
from transformers import AdamWeightDecay

batch_size = 8

data_collator = DataCollatorForLanguageModeling(mlm=False, tokenizer=tokenizer, return_tensors='tf')

tokenized_character_hg = character_hg.map(preprocess_function, batched=False)

encoded_test_set = tokenized_character_hg["test"].to_tf_dataset(
    columns=["input_ids", "attention_mask", "labels"],
    shuffle=False,
    batch_size=batch_size,
    collate_fn=data_collator,
)

Loading cached processed dataset at C:\Users\david\Documents\unibo\natural_language_processing\project\BarneyBot\cache\csv\default-fbb5da8195eeafdf\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519\cache-57294622ea31622c.arrow
Loading cached processed dataset at C:\Users\david\Documents\unibo\natural_language_processing\project\BarneyBot\cache\csv\default-fbb5da8195eeafdf\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519\cache-9abd6d9399deb390.arrow
Loading cached processed dataset at C:\Users\david\Documents\unibo\natural_language_processing\project\BarneyBot\cache\csv\default-fbb5da8195eeafdf\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519\cache-b16222792f5f5613.arrow


# Metrics Preparation

In [198]:
sample_questions = character_hg['test']['context/0']

In [199]:
n_beams = 3
top_k = 50
top_p = 0.92

def get_predictions_cached(sample_questions, model, filename, generation_method, override_predictions=False):
    prediction_path = os.path.join(in_folder, filename)
    if os.path.exists(prediction_path) and not override_predictions:
        print("Loading predictions from stored file")
        with open(prediction_path, 'r') as file:
            json_string = file.read()
        predictions = json.loads(json_string)
        print("Loaded predictions from stored file")

    else:
        print("Creating predictions")
        predictions = list()
        for x in tqdm(sample_questions):
            tokenized_question = tokenizer.encode(x + tokenizer.eos_token, return_tensors='tf')
            max_length = 128 + tokenized_question.shape[1]
            if generation_method == "Greedy":
                generated_answer = model.generate(tokenized_question,
                                    pad_token_id=tokenizer.eos_token_id, max_length=max_length)[0].numpy().tolist()
            elif generation_method == "Beam Search":
                generated_answer = model.generate(tokenized_question,
                                             pad_token_id=tokenizer.eos_token_id, max_length=max_length,
                                             n_beams=n_beams)[0].numpy().tolist()
            elif generation_method == "Sampling":
                b = True
                c = 0
                while b:
                    generated_answer = model.generate(tokenized_question,
                                                 pad_token_id=tokenizer.eos_token_id, max_length=max_length,
                                                 do_sample=True, top_k=top_k, top_p=top_p)[0].numpy().tolist()
                    c += 1
                    if len(generated_answer[len(tokenized_question[0]):])>1:
                        b = False       
                    if c>100: 
                        generated_answer[len(tokenized_question[0]):] = tokenizer.encode('hi') + [tokenizer.eos_token_id]
                        break
            
            predictions.append(generated_answer[len(tokenized_question[0]):])

        # Save predictions as a JSON file
        output_string = json.dumps(predictions)
        with open(prediction_path, 'w') as file:
            file.write(output_string)
        
        assert all([len(p)>1 for p in predictions])
        
    return predictions

In [200]:
predictions_greedy = get_predictions_cached(sample_questions, model,
                                            character_dict[character]['prediction_filename'] + '_greedy.json',
                                            "Greedy",
                                            override_predictions=override_predictions)
predictions_nbeams = get_predictions_cached(sample_questions, model,
                                            character_dict[character]['prediction_filename'] + '_nbeams.json',
                                            "Beam Search",
                                            override_predictions=override_predictions)
predictions_sampling = get_predictions_cached(sample_questions, model,
                                              character_dict[character]['prediction_filename'] + '_sampling.json',
                                              "Sampling",
                                              override_predictions=override_predictions)

Loading predictions from stored file
Loaded predictions from stored file
Loading predictions from stored file
Loaded predictions from stored file
Loading predictions from stored file
Loaded predictions from stored file


In [201]:
def get_dataframe_for_metrics(data_test, predictions_greedy, predictions_nbeams, predictions_sampling):
    i = 0
    df = {'ctx':[], 'ctx_tk':[]}
    has_labels = 'response' in data_test.features
    if has_labels:
        df['lbl'] = []
        df['lbl_tk'] = []
    if predictions_greedy:
        df['prd_greedy'] = []
        df['prd_greedy_tk'] = []
    if predictions_nbeams:
        df['prd_nbeams'] = []
        df['prd_nbeams_tk'] = [] 
    if predictions_sampling:
        df['prd_sampling'] = []
        df['prd_sampling_tk'] = []
    for sample in tqdm(data_test):
        # encode the context and label sentences, add the eos_token and return a tensor
        ctx_tk = tokenizer.encode(sample['context/0'] + tokenizer.eos_token, return_tensors='tf').numpy().tolist()
        ctx = sample['context/0']
        df['ctx_tk'].append(ctx_tk)
        df['ctx'].append(ctx)
        if has_labels:
            lbl_tk = tokenizer.encode(sample['response'] + tokenizer.eos_token, return_tensors='tf').numpy().tolist()
            lbl = sample['response']
            df['lbl'].append(lbl)
            df['lbl_tk'].append(lbl_tk)
        if predictions_greedy:
            prd_greedy_tk = predictions_greedy[i]
            prd_greedy = tokenizer.decode(prd_greedy_tk, skip_special_tokens=True)
            df['prd_greedy'].append(prd_greedy)
            df['prd_greedy_tk'].append(prd_greedy_tk)
        if predictions_nbeams:
            prd_nbeams_tk = predictions_nbeams[i]
            prd_nbeams = tokenizer.decode(prd_nbeams_tk, skip_special_tokens=True)
            df['prd_nbeams'].append(prd_nbeams)
            df['prd_nbeams_tk'].append(prd_nbeams_tk)
        if predictions_sampling:
            prd_sampling_tk = predictions_sampling[i]
            prd_sampling = tokenizer.decode(prd_sampling_tk, skip_special_tokens=True)
            df['prd_sampling'].append(prd_sampling)
            df['prd_sampling_tk'].append(prd_sampling_tk)
        i += 1
    return pd.DataFrame(data=df)

In [202]:
df_char = get_dataframe_for_metrics(character_hg['test'], predictions_greedy, predictions_nbeams, predictions_sampling)
df_char

100%|████████████████████████████████████████████████████████████████████████████| 1170/1170 [00:00<00:00, 1724.10it/s]


,ctx,ctx_tk,lbl,lbl_tk,prd_greedy,prd_greedy_tk,prd_nbeams,prd_nbeams_tk,prd_sampling,prd_sampling_tk
0,"Oh, sweetie, Im sorry.","[[5812, 11, 6029, 494, 11, 1846, 7926, 13, 502...",She called me dumbass.,"[[3347, 1444, 502, 13526, 562, 13, 50256]]","No, its okay.","[2949, 11, 663, 8788, 13, 50256]","No, its okay.","[2949, 11, 663, 8788, 13, 50256]",No. Its okay.,"[2949, 13, 6363, 8788, 13, 50256]"
1,"Oh. Wow. Good job. Okay, um, can you do this?","[[5812, 13, 24755, 13, 4599, 1693, 13, 16805, ...",Well never know.,"[[5779, 1239, 760, 13, 50256]]",I dont know.,"[40, 17666, 760, 13, 50256]",I dont know.,"[40, 17666, 760, 13, 50256]",No.,"[2949, 13, 50256]"
2,"Thanks. And, Sheldon, I know tonights the nigh...","[[9690, 13, 843, 11, 34536, 11, 314, 760, 5680...","Oh, you shouldnt have.","[[5812, 11, 345, 815, 429, 423, 13, 50256]]",Thats very impressive.,"[817, 1381, 845, 8036, 13, 50256]",Thats very impressive.,"[817, 1381, 845, 8036, 13, 50256]",Did you even ask for the ingredients?,"[11633, 345, 772, 1265, 329, 262, 9391, 30, 50..."
3,Too bad.,"[[23307, 2089, 13, 50256]]",Youre going up Euclid Avenue?,"[[1639, 260, 1016, 510, 48862, 312, 8878, 30, ...","You know, I, I, I think I, I, I think I, I thi...","[1639, 760, 11, 314, 11, 314, 11, 314, 892, 31...","You know, I, I, I think I, I, I think I, I thi...","[1639, 760, 11, 314, 11, 314, 11, 314, 892, 31...","Im coming, Penny. Oh, look who it is.","[3546, 2406, 11, 25965, 13, 220, 3966, 11, 804..."
4,"Okay, so youd lose most of Jar Jar, all the tr...","[[16454, 11, 523, 345, 67, 4425, 749, 286, 153...",Get rid of the trade route part? Then how woul...,"[[3855, 5755, 286, 262, 3292, 6339, 636, 30, 3...",I dont think I can go through with this.,"[40, 17666, 892, 314, 460, 467, 832, 351, 428,...",I dont think I can go through with this.,"[40, 17666, 892, 314, 460, 467, 832, 351, 428,...","Oh yeah, thats not bad. I love it.","[5812, 10194, 11, 29294, 407, 2089, 13, 314, 1..."
...,...,...,...,...,...,...,...,...,...,...
1165,Im available for experimentation.,"[[3546, 1695, 329, 29315, 13, 50256]]",Thank you. Not necessary. We know everything t...,"[[10449, 345, 13, 1892, 3306, 13, 775, 760, 22...","Oh, great. Well, good luck.","[5812, 11, 1049, 13, 3894, 11, 922, 8458, 13, ...","Oh, great. Well, good luck.","[5812, 11, 1049, 13, 3894, 11, 922, 8458, 13, ...","Oh. Well, see you in a while.","[5812, 13, 3894, 11, 766, 345, 287, 257, 981, ..."
1166,"Sheldon, why are you ignoring your sister?","[[3347, 25900, 11, 1521, 389, 345, 15482, 534,...",Im not ignoring my sister. Im ignoring all of ...,"[[3546, 407, 15482, 616, 6621, 13, 1846, 15482...",I dont follow.,"[40, 17666, 1061, 13, 50256]",I dont follow.,"[40, 17666, 1061, 13, 50256]",I dont think Penny was ignoring you.,"[40, 17666, 892, 25965, 373, 15482, 345, 13, 5..."
1167,"Look, Sheldon, before you race off to the fert...","[[8567, 11, 34536, 11, 878, 345, 3234, 572, 28...",You mean dating?,"[[1639, 1612, 10691, 30, 50256]]","Oh, gee, I dont know. Maybe.","[5812, 11, 308, 1453, 11, 314, 17666, 760, 13,...","Oh, gee, I dont know. Maybe.","[5812, 11, 308, 1453, 11, 314, 17666, 760, 13,...","Oh, that is a good idea.","[5812, 11, 326, 318, 257, 922, 2126, 13, 50256]"
1168,"Sheldon, Im sorry. Ill be happy to reimburse y...","[[3347, 25900, 11, 1846, 7926, 13, 5821, 307, ...",You dont need to reimburse me because Im not p...,"[[1639, 17666, 761, 284, 25903, 502, 780, 1846...","Oh, I dont mind. I just dont want to be respon...","[5812, 11, 314, 17666, 2000, 13, 314, 655, 176...","Oh, I dont mind. I just dont want to be respon...","[5812, 11, 314, 17666, 2000, 13, 314, 655, 176...",Its not fair.,"[20459, 407, 3148, 13, 50256]"


# Metrics For Character 1

In [203]:
def ccl_sim(ctx_lbl, ctx_cht, lbl_cht):
    return ((1 - abs(ctx_lbl - ctx_cht))**2 + lbl_cht**2) / 2

In [204]:
from Lib.BBMetrics import BBMetric

def compute_set_metrics(model, model_2, character, character_2, test_set_name,
                        context_sentences, label_responses, chatbot_responses, encoded_test_set,
                        classifier_n_sentences=50, label_chatbot_symmetry=False,
                        include_qualitative_sentences=False, verbose=True):
    scores = {}
    
    lbl_text = 'label' if not label_chatbot_symmetry else 'chatbota'
    cht_text = 'chatbot' if not label_chatbot_symmetry else 'chatbotb'
    
    scores['metadata'] = {}
    scores['metadata']['dataset name'] = test_set_name
    scores['metadata']['names'] = {
        'context':'context'
    }
    if label_chatbot_symmetry:
        scores['metadata']['names'][lbl_text] = character
        scores['metadata']['names'][cht_text] = character_2
    else:
        scores['metadata']['names'][lbl_text] = 'label'
        scores['metadata']['names'][cht_text] = character
    
    # 0) computes metrics for perplexity
    metric = BBMetric.load_metric("semantic similarity")
    scores['semantic similarity'] = [metric.compute(sentences_a=context_sentences,
                                            sentences_b=label_responses)]
    scores['semantic similarity'].append(metric.compute(sentences_a=context_sentences,
                                            sentences_b=chatbot_responses)),
    scores['semantic similarity'].append(metric.compute(sentences_a=label_responses,
                                              sentences_b=chatbot_responses))
    scores['semantic similarity'].append(ccl_sim(scores['semantic similarity'][0]['score'],
                                                 scores['semantic similarity'][1]['score'],
                                                 scores['semantic similarity'][2]['score']))
    scores['metadata']['semantic similarity'] = {
        'ordering': ['context-'+lbl_text, 'context-'+cht_text, cht_text+'-'+lbl_text, 'ccl']
    }
    if verbose:
        print('=== SEMANTIC SIMILARITY ===')
        print('context-'+lbl_text+' similarity:   ', scores['semantic similarity'][0])
        print('context-'+cht_text+' similarity: ', scores['semantic similarity'][1])
        print(cht_text+'-'+lbl_text+' similarity:   ', scores['semantic similarity'][2])
        print('ccl-sim similarity:            ', scores['semantic similarity'][3])
    # 1) computes metrics for perplexity
    if encoded_test_set is not None:
        metric = BBMetric.load_metric("perplexity")
        if not label_chatbot_symmetry:
            scores['perplexity'] = metric.compute(model=model, encoded_test_set=encoded_test_set)['score']
            scores['metadata']['perplexity'] = {
                'ordering': cht_text
            }
        else:
            scores['perplexity'] = [metric.compute(model=model, encoded_test_set=encoded_test_set)['score']]
            scores['perplexity'].append(metric.compute(model=model_2, encoded_test_set=encoded_test_set)['score'])
            scores['metadata']['perplexity'] = {
                'ordering': [lbl_text, cht_text]
            }
        if verbose:
            print('===       PERPLEXITY     ===')
            if label_chatbot_symmetry:
                print(lbl_text + ' perplexity:         ', scores['perplexity'][0])
                print(cht_text + ' perplexity:         ', scores['perplexity'][1])
            else:
                print(cht_text + ' perplexity:         ', scores['perplexity'])
    elif verbose:
        print("encoded_test_set not provided, skipping Perplexity.")
    # 2) computes metrics for bleu
    metric = BBMetric.load_metric("bleu")
    scores['bleu'] = [metric.compute(predictions=label_responses, references=context_sentences)]
    scores['bleu'].append(metric.compute(predictions=chatbot_responses, references=context_sentences))
    scores['bleu'].append(metric.compute(predictions=chatbot_responses, references=label_responses))
    scores['bleu'].append(ccl_sim(scores['bleu'][0]['score'],
                                  scores['bleu'][1]['score'],
                                  scores['bleu'][2]['score']))
    scores['metadata']['bleu'] = {
        'ordering': ['context-'+lbl_text, 'context-'+cht_text, cht_text+'-'+lbl_text, 'ccl']
    }
    if verbose:
        print('===         BLEU         ===')
        print('context-to-'+lbl_text+' bleu:      ', scores['bleu'][0])
        print('context-to-'+cht_text+' bleu:    ', scores['bleu'][1])
        print(lbl_text+'-to-'+cht_text+' bleu:      ', scores['bleu'][2])
        print('ccl-sim bleu:            ', scores['bleu'][3])
    # 3) computes metrics for rouge-L
    metric = BBMetric.load_metric("rouge l")
    scores['rouge l'] = [metric.compute(predictions=label_responses, references=context_sentences)]
    scores['rouge l'].append(metric.compute(predictions=chatbot_responses, references=context_sentences))
    scores['rouge l'].append(metric.compute(predictions=chatbot_responses, references=label_responses))
    scores['rouge l'].append(ccl_sim(scores['rouge l'][0]['score'],
                                     scores['rouge l'][1]['score'],
                                     scores['rouge l'][2]['score']))
    scores['metadata']['rouge l'] = {
        'ordering': ['context-'+lbl_text, 'context-'+cht_text, cht_text+'-'+lbl_text, 'ccl']
    }
    if verbose:
        print('===        ROUGE-L       ===')
        print('context-to-'+lbl_text+' rouge:     ', scores['rouge l'][0])
        print('context-to-'+cht_text+' rouge:   ', scores['rouge l'][1])
        print(lbl_text+'-to-'+cht_text+' rouge:     ', scores['rouge l'][2])
        print('ccl-sim rouge:            ', scores['rouge l'][3])
    # 4) computes metrics for distinct
    metric = BBMetric.load_metric("distinct")
    scores['distinct'] = [metric.compute(sentences=context_sentences)]
    scores['distinct'].append(metric.compute(sentences=label_responses))
    scores['distinct'].append(metric.compute(sentences=chatbot_responses))
    scores['metadata']['distinct'] = {
        'ordering': ['context', lbl_text, cht_text]
    }
    if verbose:
        print('===       DISTINCT      ===')
        print('context distinct:          ', scores['distinct'][0])
        print(lbl_text+' distinct:          ', scores['distinct'][1])
        print(cht_text+' distinct:          ', scores['distinct'][2])
        
    # 6) computes emotion metric
    metric = BBMetric.load_metric("emotion")
    scores['emotion'] = [metric.compute(sentences=context_sentences)]
    scores['emotion'].append(metric.compute(sentences=label_responses))
    scores['emotion'].append(metric.compute(sentences=chatbot_responses))
    scores['emotion'].append(sp.stats.stats.pearsonr(scores['emotion'][1]['score'],
                                                     scores['emotion'][2]['score'])[0])
    scores['metadata']['emotion'] = {
        'ordering': ['context-'+lbl_text, 'context-'+cht_text, cht_text+'-'+lbl_text, cht_text+'-'+lbl_text+' correlation']
    }
    if verbose:
        print('===       EMOTION       ===')
        print('context emotions:            \n', list(zip(scores['emotion'][0]['label'], scores['emotion'][0]['score'])))
        print(lbl_text+' emotions:              \n', list(zip(scores['emotion'][1]['label'], scores['emotion'][1]['score'])))
        print(cht_text+' emotions:            \n', list(zip(scores['emotion'][2]['label'], scores['emotion'][2]['score'])))
        print(lbl_text+'-'+cht_text+'emotion corr:  \n', scores['emotion'][3])
    # 8) computes sas metric
    metric = BBMetric.load_metric("semantic answer similarity")
    scores['semantic answer similarity'] = [metric.compute(predictions=context_sentences,
                                                    references=label_responses)]
    scores['semantic answer similarity'].append(metric.compute(predictions=context_sentences,
                                                        references=chatbot_responses))
    scores['semantic answer similarity'].append(metric.compute(predictions=label_responses,
                                                        references=chatbot_responses))
    scores['semantic answer similarity'].append(ccl_sim(scores['semantic answer similarity'][0]['score'],
                                                        scores['semantic answer similarity'][1]['score'],
                                                        scores['semantic answer similarity'][2]['score']))
    scores['metadata']['semantic answer similarity'] = {
        'ordering': ['context-'+lbl_text, 'context-'+cht_text, cht_text+'-'+lbl_text, 'ccl']
    }
    if verbose:
        print('===         SAS         ===')
        print('context-'+lbl_text+' sas:          ', scores['semantic answer similarity'][0])
        print('context-'+cht_text+' sas:        ', scores['semantic answer similarity'][1])
        print(lbl_text+'-'+cht_text+' sas:          ', scores['semantic answer similarity'][2])
        print('ccl-sim sas:               ', scores['semantic answer similarity'][3])
    # 9) computes metrics for semantic classifier
    metric = BBMetric.load_metric("semantic classifier")
    start_time = time.time()
    scores['semantic classifier'] = [metric.compute(character=character, character_dict=character_dict, 
                                                   base_folder=base_folder, sentences=label_responses,
                                                   n_sentences=classifier_n_sentences)]
    scores['semantic classifier'].append(metric.compute(character=character, character_dict=character_dict, 
                                                   base_folder=base_folder, sentences=chatbot_responses,
                                                   n_sentences=classifier_n_sentences))
    end_time = time.time()
    scores['metadata']['semantic classifier'] = {
        'ordering': [lbl_text, cht_text]
    }
    if verbose:
        print('=== SEMANTIC CLASSIFIER ===')
        print('sem-classifier '+lbl_text+':                ', scores['semantic classifier'][0])
        print('sem-classifier '+cht_text+':                  ', scores['semantic classifier'][1])
        print('time elapsed computing semantic classifier:  {:.2f} s'.format(end_time - start_time))
    if not label_chatbot_symmetry and os.path.exists(os.path.join(os.getcwd(), "Data", "Characters", character, "humancoherence.csv")):
        scores['human'] = {}
        metric = BBMetric.load_metric("human - coherence")
        scores['human']['coherence'] = metric.compute(filepath=os.path.join(os.getcwd(), "Data", "Characters",
                                                                            character, "humancoherence.csv"))
        metric = BBMetric.load_metric("human - style")
        scores['human']['style'] = metric.compute(filepath=os.path.join(os.getcwd(), "Data", "Characters",
                                                                        character, "humanstyle.csv"))
        metric = BBMetric.load_metric("human - consistency")
        scores['human']['consistency'] = metric.compute(filepath=os.path.join(os.getcwd(), "Data", "Characters",
                                                                              character, "humanconsistency.csv"))
        scores['metadata']['human'] = {
            'ordering': {
                'coherence': cht_text,
                'consistency': cht_text,
                'style': cht_text
            }
        }
        if verbose:
            print('===    HUMAN METRICS    ===')
            print('coherence:                 ', scores['human']['coherence'])
            print('consistency:               ', scores['human']['consistency'])
            print('style:                     ', scores['human']['style'])
    elif verbose:
        print("Symmetric mode, skipping Human metrics.")
    if include_qualitative_sentences:
        sentences_df = {}
        sentences_df['context'] = context_sentences
        sentences_df[lbl_text] = label_responses
        sentences_df[cht_text] = chatbot_responses
        scores['sentences'] = sentences_df
        if verbose:
            print('===      SENTENCES      ===')
            for i in range(len(context_sentences)):
                print("* context: ", context_sentences[i])
                print("* " + lbl_text + ":", label_responses[i])
                print("* " + cht_text + ":", chatbot_responses[i])
                print()
    elif verbose:
        print("Skipping sentence outputting.")
    return scores

In [ ]:
"""
set_size = 10
i = 30
print("##### Set (Size " + str(set_size) + ") #####")
context_sentences = list(df_char['ctx'][i:i+set_size])
chatbot_responses = list(df_char['prd_greedy'][i:i+set_size])
label_responses   = list(df_char['lbl'][i:i+set_size])
compute_set_metrics(model, None,
                    context_sentences, label_responses, chatbot_responses, character, encoded_test_set)
"""

In [ ]:
print("##### Full Test Set #####")
context_sentences = list(df_char['ctx'])
chatbot_responses = list(df_char['prd_greedy'])
label_responses   = list(df_char['lbl'])
scores = compute_set_metrics(model, None,
                             character, None, character + " dataset",
                             context_sentences, label_responses, chatbot_responses, encoded_test_set,
                             classifier_n_sentences=75)

In [ ]:
save_as_json(metrics_folder, character+'_base_metrics', scores)

# Metrics Between Different Sampling Methods

In [ ]:
scores = {}

In [ ]:
split = True

In [ ]:
print("##### Greedy vs. N-Beams #####")
context_sentences = list(df_char['ctx'])
greedy_responses  = list(df_char['prd_greedy'])
nbeams_responses  = list(df_char['prd_nbeams'])
scores['greedy_vs_nbeams'] = compute_set_metrics(None, None,
                                                 character, character, character + " dataset",
                                                 context_sentences,
                                                 greedy_responses,
                                                 nbeams_responses,
                                                 None,
                                                 classifier_n_sentences=75, label_chatbot_symmetry=True)

In [ ]:
if split == True:
    save_as_json(metrics_folder, character+'_greedy_vs_nbeams_metrics', scores['greedy_vs_nbeams'])

In [ ]:
print("##### Greedy vs. Sampling #####")
context_sentences   = list(df_char['ctx'])
greedy_responses    = list(df_char['prd_greedy'])
sampling_responses  = list(df_char['prd_sampling'])
scores['greedy_vs_sampling'] = compute_set_metrics(None, None,
                                                   character, character, character + " dataset",
                                                   context_sentences,
                                                   greedy_responses,
                                                   sampling_responses,
                                                   None,
                                                   classifier_n_sentences=75, label_chatbot_symmetry=True)

In [ ]:
if split == True:
    save_as_json(metrics_folder, character+'_greedy_vs_sampling_metrics', scores['greedy_vs_sampling'])

In [ ]:
print("##### N-Beams vs. Sampling #####")
context_sentences   = list(df_char['ctx'])
nbeams_responses    = list(df_char['prd_nbeams'])
sampling_responses  = list(df_char['prd_sampling'])
scores['nbeams_vs_sampling'] = compute_set_metrics(None, None,
                                                   character, character, character + " dataset",
                                                   context_sentences,
                                                   nbeams_responses,
                                                   sampling_responses,
                                                   None,
                                                   classifier_n_sentences=75, label_chatbot_symmetry=True)

In [ ]:
if split == True:
    save_as_json(metrics_folder, character+'_nbeams_vs_sampling_metrics', scores['nbeams_vs_sampling'])

In [ ]:
if split == True:    
    scores = {}
    scores['greedy_vs_nbeams'] = load_from_json(
        filepath=metrics_folder,
        filename=character+'_greedy_vs_nbeams_metrics'
    )
    scores['greedy_vs_sampling'] = load_from_json(
        filepath=metrics_folder,
        filename=character+'_greedy_vs_sampling_metrics'
    )
    scores['nbeams_vs_sampling'] = load_from_json(
        filepath=metrics_folder,
        filename=character+'_nbeams_vs_sampling_metrics'
    )
    
    os.remove(os.path.join(
        metrics_folder,
        character+'_greedy_vs_nbeams_metrics.json'
    ))
    os.remove(os.path.join(
        metrics_folder,
        character+'_greedy_vs_sampling_metrics.json'
    ))
    os.remove(os.path.join(
        metrics_folder,
        character+'_nbeams_vs_sampling_metrics.json'
    ))

In [ ]:
save_as_json(metrics_folder, character+'_sampling_comparison_metrics', scores)

# Metrics Between Character vs Non-Finetuned

In [ ]:
predictions_def_sampling = get_predictions_cached(sample_questions, model_def,
                                                  os.path.join(in_folder_def, 'from_' + character + '_df_' + '_sampling.json'),
                                                  "Sampling", override_predictions=override_predictions)

In [ ]:
df_char_def = get_dataframe_for_metrics(character_hg['test'], None, None, predictions_def_sampling)

In [ ]:
"""
for i in range(1):
    print("##### Sample " + str(i+1) + " #####")
    context_sentence   = df_char['ctx'][i]
    character_response = df_char['prd_sampling'][i]
    default_response   = df_char_def['prd_sampling'][i]
    compute_sample_metrics(context_sentence, default_response, character_response, label_chatbot_symmetry=True)
    print()
"""

In [ ]:
"""
set_size = 50
i = 30
print("##### Set (Size " + str(set_size) + ") #####")
context_sentences   = list(df_char['ctx'][i:i+set_size])
character_responses = list(df_char['prd_sampling'][i:i+set_size])
default_responses   = list(df_char_def['prd_sampling'][i:i+set_size])
compute_set_metrics(None, None,
                    context_sentences, default_responses, character_responses, character, label_chatbot_symmetry=True)
"""

In [ ]:
print("##### Full Test Set #####")
context_sentences   = list(df_char['ctx'])
character_responses = list(df_char['prd_sampling'])
default_responses   = list(df_char_def['prd_sampling'])
scores = compute_set_metrics(model, model_def, character, 'Default', character + " dataset",
                             context_sentences, 
                             character_responses, 
                             default_responses,
                             encoded_test_set,
                             classifier_n_sentences=75,
                             label_chatbot_symmetry=True)

In [ ]:
save_as_json(metrics_folder, character+'_vs_nonfinetuned_metrics', scores)

# Metrics Between Character 1 & Character 2

In [205]:
def get_predictions_small(sample_questions, model, generation_method):
    print("Creating predictions")
    predictions = list()
    for x in tqdm(sample_questions):
        tokenized_question = tokenizer.encode(x + tokenizer.eos_token, return_tensors='tf')
        max_length = 128 + tokenized_question.shape[1]
        if generation_method == "Greedy":
            generated_answer = model.generate(tokenized_question,
                                pad_token_id=tokenizer.eos_token_id, max_length=max_length)[0].numpy().tolist()
        elif generation_method == "Beam Search":
            generated_answer = model.generate(tokenized_question,
                                         pad_token_id=tokenizer.eos_token_id, max_length=max_length,
                                         n_beams=n_beams)[0].numpy().tolist()
        elif generation_method == "Sampling":
                b = True
                c = 0
                while b:
                    generated_answer = model.generate(tokenized_question,
                                                 pad_token_id=tokenizer.eos_token_id, max_length=max_length,
                                                 do_sample=True, top_k=top_k, top_p=top_p)[0].numpy().tolist()
                    
                    c+= 1
                    if len(generated_answer[len(tokenized_question[0]):])>1:
                        b = False         
                    if c>100: 
                        generated_answer[len(tokenized_question[0]):] = tokenizer.encode('hi') + [tokenizer.eos_token_id]
                        break 
                        
        predictions.append(generated_answer[len(tokenized_question[0]):])
        
        assert all([len(p)>1 for p in predictions])
        
    return predictions

In [206]:
df_common = load_dataset('csv',
                         data_files=os.path.join(base_folder, 'Data', 'common_dataset.csv'), 
                         cache_dir=os.path.join(base_folder, "cache"))

df_common = df_common.remove_columns(['source'])
tokenized_common_hg = df_common['train'].map(preprocess_function, batched=False)

encoded_common_set = tokenized_common_hg.to_tf_dataset(
    columns=["input_ids", "attention_mask", "labels"],
    shuffle=False,
    batch_size=batch_size,
    collate_fn=data_collator,
)

Using custom data configuration default-4624fb5057215352
Reusing dataset csv (C:\Users\david\Documents\unibo\natural_language_processing\project\BarneyBot\cache\csv\default-4624fb5057215352\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached processed dataset at C:\Users\david\Documents\unibo\natural_language_processing\project\BarneyBot\cache\csv\default-4624fb5057215352\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519\cache-0024dfc524fce5dc.arrow


In [207]:
df_common

DatasetDict({
    train: Dataset({
        features: ['label', 'context/0'],
        num_rows: 35
    })
})

In [208]:
encoded_common_set

<PrefetchDataset element_spec={'input_ids': TensorSpec(shape=(None, None), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(None, None), dtype=tf.int64, name=None), 'labels': TensorSpec(shape=(None, None), dtype=tf.int64, name=None)}>

In [209]:
predictions_1_sampling = get_predictions_small(df_common['train']['context/0'], model, "Sampling")

Creating predictions


100%|██████████████████████████████████████████████████████████████████████████████████| 35/35 [00:51<00:00,  1.47s/it]


In [210]:
predictions_2_sampling = get_predictions_small(df_common['train']['context/0'], model_2, "Sampling")

Creating predictions


100%|██████████████████████████████████████████████████████████████████████████████████| 35/35 [00:55<00:00,  1.58s/it]


In [211]:
df_common_char_1 = get_dataframe_for_metrics(df_common['train'], None, None, predictions_1_sampling)
df_common_char_2 = get_dataframe_for_metrics(df_common['train'], None, None, predictions_2_sampling)

100%|████████████████████████████████████████████████████████████████████████████████| 35/35 [00:00<00:00, 2914.10it/s]


In [212]:
print("##### " + character + "  Vs. " + character_2 + " #####")
context_sentences   = list(df_common_char_1['ctx'])
chatbot_responses   = list(df_common_char_1['prd_sampling'])
chatbot_2_responses = list(df_common_char_2['prd_sampling'])
scores = compute_set_metrics(model, model_2, character, character_2, "common small dataset",
                             context_sentences, chatbot_responses, chatbot_2_responses, encoded_common_set,
                             include_qualitative_sentences=True, label_chatbot_symmetry=True)

##### Sheldon  Vs. Barney #####
=== SEMANTIC SIMILARITY ===
context-chatbota similarity:    {'score': 0.3438683748245239, 'std': 0.20891159772872925}
context-chatbotb similarity:  {'score': 0.3270496428012848, 'std': 0.15701977908611298}
chatbotb-chatbota similarity:    {'score': 0.2786218225955963, 'std': 0.1434633731842041}
ccl-sim similarity:             0.5221377628634416


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:04<00:00,  1.23it/s]


===       PERPLEXITY     ===
chatbota perplexity:          67.96138565177583
chatbotb perplexity:          60.62977505345756
===         BLEU         ===
context-to-chatbota bleu:       {'score': 0.0}
context-to-chatbotb bleu:     {'score': 0.010537053648441798}
chatbota-to-chatbotb bleu:       {'score': 0.0}
ccl-sim bleu:             0.4895184611013533
===        ROUGE-L       ===
context-to-chatbota rouge:      {'score': 0.14642306237639446, 'std': 0.2228028221728049}
context-to-chatbotb rouge:    {'score': 0.0682674913445552, 'std': 0.11311735470852449}
chatbota-to-chatbotb rouge:      {'score': 0.05256667602067202, 'std': 0.06728318343210075}
ccl-sim rouge:             0.42628020332374833
===       DISTINCT      ===
context distinct:           {'score': 0.10984328048288133, 'std': 0.06188826948933667}
chatbota distinct:           {'score': 0.13308285410006637, 'std': 0.06287864172395086}
chatbotb distinct:           {'score': 0.12777674962949512, 'std': 0.056132893489592085}
===   

In [213]:
save_as_json(metrics_folder, character+'_vs_'+character_2+'_metrics', scores)